In [4]:
import pandas as pd 
import numpy as np 
import os 
import json 

#raw_path = "/home/ihsan/Documents/thesis_generator/results/devin/to_process/" #needs the absolute path, no tildes! 
#processed_path = "/home/ihsan/Documents/thesis_generator/results/devin"

#usb drive
#raw_path = '/media/ihsan/LID_FLASH_1/Thesis/thesis_generator/results/run_2/'

raw_path = "/home/ihsan/Documents/thesis_generator/results/to_process/"

#processed_path = "/home/ihsan/Documents/thesis_models/unsplit" #my poor SSD. 
#processed_path = '/media/ihsan/LID_FLASH_1/Thesis/thesis_generator/results/run_2/processed/'
processed_path = '/media/ihsan/BigRigData/Thesis/unsplit/'
items = os.listdir(raw_path)
items.sort()
print(type(items))
for file in items:
    if ('.csv') not in str(file):
        del items[items.index(file)]   
print(items)

feature_version = "_fv1b" #1b: still crack tip position, but with percent damage added. 
features_list = ['percent_damage','delta_K_current_1','ctip_posn_curr_1','delta_K_current_2','ctip_posn_curr_2',
          'delta_K_current_3','ctip_posn_curr_3','delta_K_current_4','ctip_posn_curr_4','Load_1','Load_2']

seq_length_dict = {}
seq_length_dict_filename = processed_path + "/sequence_lengths" + feature_version + "_.json"
#suffix = "3a"
#csv_path = "~/Documents/thesis_generator/results/devin/crack_growth_sequence" + suffix + ".csv"
sequence_lengths = {} #save sequence lengths as a dict. or maybe a json?

<type 'list'>
['crack_growth_sequence_1d_0.csv', 'crack_growth_sequence_1d_1.csv', 'crack_growth_sequence_1d_2.csv', 'crack_growth_sequence_1d_3.csv', 'crack_growth_sequence_1d_4.csv', 'crack_growth_sequence_1d_5.csv']


In [5]:
j = 0 #counter. 
threshold = 0.5 #threshold crack length. rivet pitch is 0.875" so a bit over half of that. 

for file in items:
    print("filename: {}".format(str(file)))
    csv_path = raw_path + str(file)
    if ("_0.") in str(file): #only the first file in the series has a header. 
        cg_seq_df = pd.read_csv(csv_path)
        header_names = cg_seq_df.columns.values
        print("header names: {}".format(header_names))
    else:
        cg_seq_df = pd.read_csv(csv_path,names=header_names)
        print(cg_seq_df.columns.values)
       
    cg_seq_df['percent_damage'] = \
    cg_seq_df[['crack_length_1','crack_length_2','crack_length_3','crack_length_4']].max(axis=1)/threshold * 100
    train_list = ['StepIndex','percent_damage','delta_K_current_1','ctip_posn_curr_1','delta_K_current_2','ctip_posn_curr_2',
          'delta_K_current_3','ctip_posn_curr_3','delta_K_current_4','ctip_posn_curr_4','Load_1','Load_2'] #and seq_id,somehow 

    label_list = ['StepIndex','delta_a_current_1','delta_a_current_2','delta_a_current_3','delta_a_current_4']

    train_df = cg_seq_df[train_list]
    print("cg_seq_df shape: {}".format(cg_seq_df.columns.values))
    print(train_df.index,train_df.head(1))
    
    #---------------------NEW PART, ADDS PERCENT DAMAGE----------------------------------------
#     #train_df['percent_damage'] = \
#     train_df[['crack_length_1','crack_length_2','crack_length_3','crack_length_4']].max(axis=1)/threshold * 100
#     print(train_df.head(2),train_df_play.index,train_df_play.info())
#     print("before dropping changing: {}".format(train_df.columns.values))
#     train_df = train_df[['percent_damage','delta_K_current_1','crack_length_1','delta_K_current_2',
#      'crack_length_2','delta_K_current_3','crack_length_3','delta_K_current_4',
#                                   'crack_length_4','Load_1','Load_2']]
#     print("after dropping: {}".format(train_df.columns.values))
    #------------------------------------------------------------------------------------------
    label_train_df = cg_seq_df[label_list]

    #labels_csv_path = "~/Documents/thesis_generator/results/devin/label" + suffix + "_headers"+ ".csv"
    #train_csv_path = "~/Documents/thesis_generator/results/devin/train" + suffix + "_headers" + ".csv"

    indices = train_df[train_df['StepIndex']==1].index.tolist()
    indices.append(train_df.shape[0]-1)
    indices_offset_min1 = [i-1 for i in indices]
    print("file {}'s indices_offset_min1 {}".format(str(file),indices_offset_min1))
    indices_offset_min1.pop(0)
    print("indices: {}, indices_offset_min1: {}".format(indices,indices_offset_min1))

    ranges = [(t,s) for t,s in zip(indices,indices_offset_min1)]
    #print("before changing :{}".format(ranges))
    '''for tuple in ranges:
        print(tuple)
        tuple[1:][0] = (tuple[1:][0]) + 1'''

    #ranges[1:][0] = ranges[1:][0] + 1
    print("\nafter changing :{} ".format(ranges))
    #print("lengths: {} ".format([indices[4]-indices[3],indices[3]-indices[2],indices[2]-indices[1],indices[1]-indices[0]]))
    print("lengths: {} ".format([t-s for (s,t) in ranges]))
    
    i = 0
    for indices_as_tuples in ranges:
        i=i+1
        print("indices as tuples: {}".format(indices_as_tuples))
        
        #not an np array yet.. needs to drop the stepindex. 
        train_df_as_np_array = train_df[indices_as_tuples[0]:indices_as_tuples[1]]
        column_check = all(col_names in train_df_as_np_array.columns.values for col_names in features_list) #GOTTA BE THE SAME NAME YO
        #================ drop the stepindex before casting as a numpy array===================
        train_df_as_np_array = train_df_as_np_array[['percent_damage','delta_K_current_1','ctip_posn_curr_1','delta_K_current_2','ctip_posn_curr_2',
          'delta_K_current_3','ctip_posn_curr_3','delta_K_current_4','ctip_posn_curr_4','Load_1','Load_2']]
        print("All header names saved? {}".format(column_check))
        train_df_as_np_array = train_df_as_np_array.values
        
        #label is ok as is 
        label_train_df_as_np_array = label_train_df[indices_as_tuples[0]:indices_as_tuples[1]].values
        print("df_as_np_array shape: {}".format(train_df_as_np_array.shape))
        print("file: {}".format(file))
        identifier = str(str(file)[-8:-6]) #eg 1a 2a etc. #you can use a regex. 
        print("identifier: {}".format(identifier))
        
        #j is sequence id. #i is the sequence number within the csv. #fv2 = feature version 2. 
        np_train_path = processed_path + "/sequence_" + identifier + "_" + str(j) + "_" + str(i) + feature_version + ".npy"
        np_label_train_path = processed_path + "/sequence_" + identifier + "_" + str(j) + "_" + str(i) + "_label" + feature_version + ".npy"
        seq_length_dict["sequence_" + identifier + "_" + str(j) + "_" + str(i) + feature_version] = indices_as_tuples[1] - indices_as_tuples[0]
        #seq_length_dict = json.load(open(seq_length_dict))
        #UNCOMMENT THESE IF YOU WANT TO SAVETHE ARRAYS!!!
        print("np_train_path: {}".format(np_train_path))
        print("np_label_train_path :{}".format(np_label_train_path))
        np.save(np_train_path,train_df_as_np_array)
        np.save(np_label_train_path,label_train_df_as_np_array)
        j= j + 1
        
print(seq_length_dict)
json.dump(seq_length_dict, open(seq_length_dict_filename, 'wb'))

filename: crack_growth_sequence_1d_0.csv
header names: ['Unnamed: 0' 'StepIndex' 'direction_1' 'delta_a_current_1'
 'delta_K_prev_1' 'delta_K_current_1' 'ctip_posn_curr_1' 'ctip_posn_prev_1'
 'crack_length_last_overload_1' 'crack_length_1' 'K_last_overload_1'
 'pz_rad_curr_1' 'pz_rad_prev_1' 'crack_id_1' 'hole_1' 'direction_2'
 'delta_a_current_2' 'delta_K_prev_2' 'delta_K_current_2'
 'ctip_posn_curr_2' 'ctip_posn_prev_2' 'crack_length_last_overload_2'
 'crack_length_2' 'K_last_overload_2' 'pz_rad_curr_2' 'pz_rad_prev_2'
 'crack_id_2' 'hole_2' 'direction_3' 'delta_a_current_3' 'delta_K_prev_3'
 'delta_K_current_3' 'ctip_posn_curr_3' 'ctip_posn_prev_3'
 'crack_length_last_overload_3' 'crack_length_3' 'K_last_overload_3'
 'pz_rad_curr_3' 'pz_rad_prev_3' 'crack_id_3' 'hole_3' 'direction_4'
 'delta_a_current_4' 'delta_K_prev_4' 'delta_K_current_4'
 'ctip_posn_curr_4' 'ctip_posn_prev_4' 'crack_length_last_overload_4'
 'crack_length_4' 'K_last_overload_4' 'pz_rad_curr_4' 'pz_rad_prev_4'
 'cr

All header names saved? True
df_as_np_array shape: (98360, 11)
file: crack_growth_sequence_1d_0.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_16_17_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_16_17_label_fv1b.npy
indices as tuples: (1672481, 1770811)
All header names saved? True
df_as_np_array shape: (98330, 11)
file: crack_growth_sequence_1d_0.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_17_18_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_17_18_label_fv1b.npy
indices as tuples: (1770812, 1869032)
All header names saved? True
df_as_np_array shape: (98220, 11)
file: crack_growth_sequence_1d_0.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_18_19_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_18_19_label_fv1b.npy
indices as tuples: (1869033, 1967368)
All header na

indices as tuples: (1010571, 1111569)
All header names saved? True
df_as_np_array shape: (100998, 11)
file: crack_growth_sequence_1d_1.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_30_11_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_30_11_label_fv1b.npy
indices as tuples: (1111570, 1212638)
All header names saved? True
df_as_np_array shape: (101068, 11)
file: crack_growth_sequence_1d_1.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_31_12_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_31_12_label_fv1b.npy
indices as tuples: (1212639, 1313686)
All header names saved? True
df_as_np_array shape: (101047, 11)
file: crack_growth_sequence_1d_1.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_32_13_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_32_13_label_fv1b.npy
indices as

indices as tuples: (642012, 749090)
All header names saved? True
df_as_np_array shape: (107078, 11)
file: crack_growth_sequence_1d_2.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_46_7_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_46_7_label_fv1b.npy
indices as tuples: (749091, 856185)
All header names saved? True
df_as_np_array shape: (107094, 11)
file: crack_growth_sequence_1d_2.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_47_8_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_47_8_label_fv1b.npy
indices as tuples: (856186, 963266)
All header names saved? True
df_as_np_array shape: (107080, 11)
file: crack_growth_sequence_1d_2.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_48_9_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_48_9_label_fv1b.npy
indices as tuples: (96

indices as tuples: (287751, 383651)
All header names saved? True
df_as_np_array shape: (95900, 11)
file: crack_growth_sequence_1d_3.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_63_4_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_63_4_label_fv1b.npy
indices as tuples: (383652, 479502)
All header names saved? True
df_as_np_array shape: (95850, 11)
file: crack_growth_sequence_1d_3.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_64_5_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_64_5_label_fv1b.npy
indices as tuples: (479503, 575386)
All header names saved? True
df_as_np_array shape: (95883, 11)
file: crack_growth_sequence_1d_3.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_65_6_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_65_6_label_fv1b.npy
indices as tuples: (57538

indices as tuples: (232276, 309668)
All header names saved? True
df_as_np_array shape: (77392, 11)
file: crack_growth_sequence_1d_4.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_83_4_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_83_4_label_fv1b.npy
indices as tuples: (309669, 387149)
All header names saved? True
df_as_np_array shape: (77480, 11)
file: crack_growth_sequence_1d_4.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_84_5_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_84_5_label_fv1b.npy
indices as tuples: (387150, 464614)
All header names saved? True
df_as_np_array shape: (77464, 11)
file: crack_growth_sequence_1d_4.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_85_6_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_85_6_label_fv1b.npy
indices as tuples: (46461

indices as tuples: (291157, 388222)
All header names saved? True
df_as_np_array shape: (97065, 11)
file: crack_growth_sequence_1d_5.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_103_4_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_103_4_label_fv1b.npy
indices as tuples: (388223, 485288)
All header names saved? True
df_as_np_array shape: (97065, 11)
file: crack_growth_sequence_1d_5.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_104_5_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_104_5_label_fv1b.npy
indices as tuples: (485289, 582351)
All header names saved? True
df_as_np_array shape: (97062, 11)
file: crack_growth_sequence_1d_5.csv
identifier: 1d
np_train_path: /media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_105_6_fv1b.npy
np_label_train_path :/media/ihsan/BigRigData/Thesis/unsplit//sequence_1d_105_6_label_fv1b.npy
indices as tuples: 